<a href="https://colab.research.google.com/github/gaiobr/aprendizado-de-maquina-edge-ufal/blob/master/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


## **1. Importando bibliotecas**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

## **2. Lendo imagens**

In [2]:
input_dir = '/content/drive/My Drive/Colab Notebooks/datasets/persian/'

In [3]:
#tipo = train ou test
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 32 if tipo == 'train' else 10
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file)
      print(img_file, ': ', img.shape)
      #transforme imagem em array de pixels e salve na lista de imagens      
      X.append(img)
      #guarde a classificação da imagem
      y.append(pasta)
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y

In [ ]:
X_treino,y_treino = ler_dataset('train')

In [5]:
X_treino.shape

(576, 50, 50, 3)

In [6]:
y_treino.shape

(576,)

In [ ]:
X_teste,y_teste = ler_dataset('test')

In [8]:
X_teste.shape

(180, 50, 50, 3)

In [9]:
y_teste.shape

(180,)

In [ ]:
X_treino

In [11]:
np.unique(y_treino)

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '2',
       '3', '4', '5', '6', '7', '8', '9'], dtype='<U2')

In [12]:
#transformando labels de numérico para categórico
y_treino = to_categorical(y_treino, num_classes = 19)
y_teste = to_categorical(y_teste, num_classes = 19)

In [13]:
X_treino = X_treino/255
X_teste = X_teste/255

In [ ]:
y_treino

In [15]:
#definição do modelo
model = Sequential()

# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (50,50,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 128 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

#  2 camadas fully conected 
model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(19, activation = "softmax"))

In [16]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [17]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [18]:
# Descrição do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 32)        2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        3

In [19]:
# treinamento do modelo
model.fit(X_treino, y_treino, epochs=10,verbose = 1)

Epoch 1/10
18/18 [==============================] - 25s 1s/step - loss: 3.2829 - accuracy: 0.0608
Epoch 2/10
18/18 [==============================] - 24s 1s/step - loss: 2.4732 - accuracy: 0.2691
Epoch 3/10
18/18 [==============================] - 24s 1s/step - loss: 1.4067 - accuracy: 0.5660
Epoch 4/10
18/18 [==============================] - 24s 1s/step - loss: 0.9333 - accuracy: 0.6892
Epoch 5/10
18/18 [==============================] - 24s 1s/step - loss: 0.6841 - accuracy: 0.8003
Epoch 6/10
18/18 [==============================] - 24s 1s/step - loss: 0.6633 - accuracy: 0.8090
Epoch 7/10
18/18 [==============================] - 26s 1s/step - loss: 0.4991 - accuracy: 0.8455
Epoch 8/10
18/18 [==============================] - 24s 1s/step - loss: 0.4767 - accuracy: 0.8559
Epoch 9/10
18/18 [==============================] - 24s 1s/step - loss: 0.3907 - accuracy: 0.8819
Epoch 10/10
18/18 [==============================] - 24s 1s/step - loss: 0.3360 - accuracy: 0.9045


In [20]:
model.evaluate(X_teste,y_teste)

6/6 [==============================] - 2s 256ms/step - loss: 0.1412 - accuracy: 0.9611


[0.14115358889102936, 0.9611111283302307]

## **8. Análise dos Resultados**

O conjunto de dados é um conjunto bastante pequeno.

São 18 categorias com 32 imagens cada. 

A acurácia do modelo foi baixa quando usando apenas 1 ou 2 épocas, porém aumentando o número de épocas de treinamento, o modelo conseguiu alcançar uma acurácia de 90% no treino e de 96% na avaliação.